In [ ]:
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

from joblib import dump, load

from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import confusion_matrix

from keras.callbacks import EarlyStopping
from keras.models import Model, Sequential
from keras.layers import Input, LSTM, RepeatVector, TimeDistributed, Dense, Dropout, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy

In [ ]:
sns.set_theme()
RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)  # Semente para numpy

# Carregando os dados

Os valores são números muito pequenos com muitas casas decimais, por isso é bom que o dataframe consiga representar isso também.

In [ ]:
pd.set_option('display.float_format', '{:.20f}'.format)

In [ ]:
df_benign = pd.read_csv("data/dados benignos/mensagens_benignas.csv")
df_benign

In [ ]:
df_malicious_random_dos = pd.read_csv("data/ataques/mensagens_maliciosas_random_dos.csv")
df_malicious_spoofing_zero_payload = pd.read_csv("data/ataques/mensagens_maliciosas_spoofing_zero_payload.csv")
df_malicious_zero_dos = pd.read_csv("data/ataques/mensagens_maliciosas_zero_dos.csv")

In [ ]:
df_malicious_random_dos

In [ ]:
df_malicious_spoofing_zero_payload

In [ ]:
df_malicious_zero_dos

In [ ]:
# OPCIONAL!!!
#df_benign = df_benign.head(int(len(df_benign) * 0.08))
#df_malicious_random_dos = df_malicious_random_dos.head(int(len(df_malicious_random_dos) * 0.08))
#df_malicious_spoofing_zero_payload = df_malicious_spoofing_zero_payload.head(int(len(df_malicious_spoofing_zero_payload) * 0.08))
#df_malicious_zero_dos = df_malicious_zero_dos.head(int(len(df_malicious_zero_dos) * 0.08))

# Tratando dados

## Normalização dos dados

In [ ]:
scaler_minmax = MinMaxScaler()

scaler_minmax.fit(df_benign)

df_benign_scaled = pd.DataFrame(scaler_minmax.transform(df_benign), columns=df_benign.columns, index=df_benign.index)
df_malicious_random_dos_scaled = pd.DataFrame(scaler_minmax.transform(df_malicious_random_dos), columns=df_malicious_random_dos.columns, index=df_malicious_random_dos.index)
df_malicious_spoofing_zero_payload_scaled = pd.DataFrame(scaler_minmax.transform(df_malicious_spoofing_zero_payload), columns=df_malicious_spoofing_zero_payload.columns, index=df_malicious_spoofing_zero_payload.index)
df_malicious_zero_dos_scaled = pd.DataFrame(scaler_minmax.transform(df_malicious_zero_dos), columns=df_malicious_zero_dos.columns, index=df_malicious_zero_dos.index)

In [ ]:
del df_benign
del df_malicious_random_dos
del df_malicious_spoofing_zero_payload
del df_malicious_zero_dos

In [ ]:
df_benign_scaled

## Criando Labels

In [ ]:
list_labels_benign = [1] * len(df_benign_scaled)
list_labels_random_dos = [-1] * len(df_malicious_random_dos_scaled)
list_labels_spoofing_zero_payload = [-2] * len(df_malicious_spoofing_zero_payload_scaled)
list_labels_zero_dos = [-3] * len(df_malicious_zero_dos_scaled)

## Criação de Janelas Temporais

In [ ]:
def create_slicing_windows(data, labels, time_step=1):
    X, Y = [], []
    for i in range(len(data) - time_step):
        a = data[i:(i + time_step)]
        X.append(a)
        Y.append(labels[i + time_step])
    return np.array(X), np.array(Y)

In [ ]:
WINDOW_SIZE = 100

In [ ]:
benign_windows, benign_labels = create_slicing_windows(df_benign_scaled, list_labels_benign, WINDOW_SIZE)
del df_benign_scaled, list_labels_benign

malicious_random_dos_windows, malicious_random_dos_labels = create_slicing_windows(df_malicious_random_dos_scaled, list_labels_random_dos, WINDOW_SIZE)
del df_malicious_random_dos_scaled, list_labels_random_dos

malicious_spoofing_zero_payload_windows, malicious_spoofing_zero_payload_labels = create_slicing_windows(df_malicious_spoofing_zero_payload_scaled, list_labels_spoofing_zero_payload, WINDOW_SIZE)
del df_malicious_spoofing_zero_payload_scaled, list_labels_spoofing_zero_payload

malicious_zero_dos_windows, malicious_zero_dos_labels = create_slicing_windows(df_malicious_zero_dos_scaled, list_labels_zero_dos, WINDOW_SIZE)
del df_malicious_zero_dos_scaled, list_labels_zero_dos

In [ ]:
len(benign_windows), len(benign_windows[0])

In [ ]:
benign_windows

## Dividindo dados em Treino, Validação e Teste

In [ ]:
len(benign_windows) / (len(benign_windows) + len(malicious_random_dos_windows) + len(malicious_spoofing_zero_payload_windows) + len(malicious_zero_dos_windows))

In [ ]:
data = np.vstack((benign_windows, malicious_random_dos_windows, malicious_spoofing_zero_payload_windows, malicious_zero_dos_windows))

In [ ]:
data_labels = np.hstack((benign_labels, malicious_random_dos_labels, malicious_spoofing_zero_payload_labels, malicious_zero_dos_labels))

In [ ]:
del benign_windows
del malicious_random_dos_windows
del malicious_spoofing_zero_payload_windows
del malicious_zero_dos_windows

del benign_labels
del malicious_random_dos_labels
del malicious_spoofing_zero_payload_labels
del malicious_zero_dos_labels

In [ ]:
train_data, val_test_data, train_labels, val_test_labels = train_test_split(data, data_labels, test_size=0.25, random_state=RANDOM_SEED)

In [ ]:
val_data, test_data, val_labels, test_labels = train_test_split(val_test_data, val_test_labels, test_size=0.5, random_state=RANDOM_SEED)

In [ ]:
del val_test_data, val_test_labels

In [ ]:
train_data.shape

# IAs

## LSTM

In [ ]:
FEATURES_COUNT = 7 # Número de features dos dados

In [ ]:
# Construindo o modelo LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(WINDOW_SIZE, FEATURES_COUNT)))
model.add(BatchNormalization())
model.add(Dense(1, kernel_regularizer=l2(0.000000001)))

# Compilando o modelo
model.compile(optimizer=Adam(learning_rate=0.003), loss='mse')
#model.compile(optimizer="adam", loss='mse')


In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [ ]:
# Treinando o modelo
history = model.fit(train_data, train_labels, epochs=2, batch_size=32, validation_data=(val_data, val_labels), callbacks=[early_stopping]) # padrão: batch_size=32

In [ ]:
# Avaliando o modelo no conjunto de teste
loss = model.evaluate(test_data, test_labels)
print("Test Loss:", loss)

In [ ]:
predicts = model.predict(test_data)

In [ ]:
predicts

In [ ]:
len(predicts[predicts > 0]) / (len(predicts[predicts < 0]) + len(predicts[predicts > 0]))

In [ ]:
len(test_labels[test_labels < 0]), len(test_labels[test_labels > 0])

In [ ]:
len(predicts[predicts < 0]), len(predicts[predicts > 0])

In [ ]:
#model.save('meu_modelo.h5')